# 서울시 범죄 현황 데이터 개요 확인

In [1]:
import pandas as pd
import numpy as np

#### 🔰 데이터 읽기

- `thousands` param은 단위 구분 문자(콤마)가 찍힌 숫자를 문자로 인식할 수 있기 때문에 설정으로 넣어준다.

	- 숫자값들이 콤마(,)를 사용하고 있어서 문자로 인식될 수 있다.
	- 천단위 구분(thousands=',')이라고 알려주면 컴마를 제거하고 숫자형으로 읽어온다.

In [2]:
crime_raw_data = pd.read_csv("../data/02. crime_in_Seoul.csv", thousands=',', encoding='euc-kr')
crime_raw_data.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,2.0
1,중부,살인,검거,2.0
2,중부,강도,발생,3.0
3,중부,강도,검거,3.0
4,중부,강간,발생,141.0


#### 🔰 기본 정보 확인

- `info()` 메서드로 데이터의 개요를 확인한다.

In [3]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65534 entries, 0 to 65533
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


👆 RangeIndex: 65534 entries, 0 to 65533 </br>

- 자료는 310개인데 인덱스가 6만개가 넘는다.

- 특정 컬럼에서 unique 조사

In [4]:
crime_raw_data["죄종"].unique()

array(['살인', '강도', '강간', '절도', '폭력', nan], dtype=object)

👆 "죄종" 컬럼에 nan 값이 들어가 있음을 확인할 수 있다.

- `isnull()` 메서드를 이용해 Value가 Null인지 아닌지 True, False로 결과를 출력해 본다.

In [5]:
crime_raw_data["죄종"].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
65529     True
65530     True
65531     True
65532     True
65533     True
Name: 죄종, Length: 65534, dtype: bool

- crime_raw_data로 마스킹을 씌워 True인 값들만 DataFrame형태로 출력한다.

In [6]:
crime_raw_data[crime_raw_data["죄종"].isnull()].head()

,구분,죄종,발생검거,건수
310,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN
314,NaN,NaN,NaN,NaN


👆 Index [310]부터 NaN값으로 채워진 것을 확인할 수 있다.

- 이럴 때는 NaN을 제거하는 것이 아니라 NaN값을 제외하고 필요한 데이터만 가져온다.

In [7]:
crime_raw_data[crime_raw_data["죄종"].notnull()]

,구분,죄종,발생검거,건수
0,중부,살인,발생,2.0
1,중부,살인,검거,2.0
2,중부,강도,발생,3.0
3,중부,강도,검거,3.0
4,중부,강간,발생,141.0
...,...,...,...,...
305,수서,강간,검거,144.0
306,수서,절도,발생,1149.0
307,수서,절도,검거,789.0
308,수서,폭력,발생,1666.0


In [11]:
crime_raw_data = crime_raw_data[crime_raw_data["죄종"].notnull()]
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.1+ KB


👆 NaN을 정리하고 나니 데이터의 크기가 2MB에서 12.1KB가 되었다.

- head()와 tail()로 값을 찍어 문제가 없는지 확인한다.

In [12]:
crime_raw_data.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,2.0
1,중부,살인,검거,2.0
2,중부,강도,발생,3.0
3,중부,강도,검거,3.0
4,중부,강간,발생,141.0


In [13]:
crime_raw_data.tail()

,구분,죄종,발생검거,건수
305,수서,강간,검거,144.0
306,수서,절도,발생,1149.0
307,수서,절도,검거,789.0
308,수서,폭력,발생,1666.0
309,수서,폭력,검거,1431.0


- 이 데이터의 형태로는 의미있는 insight를 도출하기 어렵다.
- 형태가 세로축에 서울기 구이름이 있고, 가로축에 5대 범죄 수치가 있으면 좋을 것같다.
- 즉, 단순 나열된 데이터를 원하는 데이터 열로 재배치할 필요가 있다.

- 이것을 쉽게 해주는 것이 Pandas의 `pivot_table`이다.

In [16]:
crime_raw_data.to_csv("./result_data/01_crime_raw_data.csv", encoding='utf-8', index=False)